In [90]:
!pip install parsel requests pandas openpyxl


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import requests
import parsel
import os
import re
import time
import pandas as pd

In [127]:
def consultar_cep(cep):
    cep = ''.join(filter(str.isdigit, str(cep)))
    url = f'https://brasilapi.com.br/api/cep/v1/{cep}'
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        else:
            print(f'Erro ao consultar o CEP {cep}: Status Code {response.status_code}')
            return None
    except Exception as e:
        print(f'Erro ao consultar o CEP {cep}: {e}')
        return None

In [2]:
url = 'https://sga.softexrecife.org.br/catalogo'

resposta = requests.get(url)

In [3]:
resposta

<Response [200]>

In [4]:
resposta.text

'<!DOCTYPE html>\n<html lang="en">\n  <!-- BEGIN HEAD -->\n  <head>\n    <meta charset="utf-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\n    <meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1, shrink-to-fit=no">\n    <title>SGA - Softex Recife</title>\n\n    <!-- BEGIN GLOBAL MANDATORY STYLES -->\n    <link href="https://fonts.googleapis.com/css2?family=Sulphur+Point:wght@300&display=swap" rel="stylesheet">\n    <link href="https://fonts.googleapis.com/css2?family=Roboto:wght@100;300&display=swap" rel="stylesheet">\n    <link href="https://cdn.jsdelivr.net/npm/bootstrap-icons@1.11.0/font/bootstrap-icons.css" rel="stylesheet" >\n    <link rel="stylesheet" media="screen" href="/assets/packs/css/stylesheets/soar/pages_layout-cd65782b.css" />\n    <script src="/assets/packs/js/javascripts/application-65a8c1dec047659e4d61.js" data-turbolinks-track="reload"></script>\n    <script src="/assets/packs/js/javascripts/main-5a4776e5e62ccf4c6ff3.j

In [5]:
html_text = resposta.text

In [7]:
html_selector = parsel.Selector(html_text)

In [8]:
links = html_selector.xpath("//div[@class='catalog-content-main-company border cl-page-width rounded row d-flex align-items-center justify-content-center']/a/@href").getall()

In [9]:
print(links)
len(links)

['/catalogo/470', '/catalogo/469', '/catalogo/468', '/catalogo/471', '/catalogo/467', '/catalogo/464', '/catalogo/463', '/catalogo/462', '/catalogo/461', '/catalogo/460', '/catalogo/459', '/catalogo/457', '/catalogo/455', '/catalogo/453', '/catalogo/452', '/catalogo/450', '/catalogo/449', '/catalogo/446', '/catalogo/444', '/catalogo/448']


20

In [10]:
cards_url = []

for link in links:
    cards_url.append(url + '/' + os.path.basename(link))

print(len(cards_url))
print(cards_url)

20
['https://sga.softexrecife.org.br/catalogo/470', 'https://sga.softexrecife.org.br/catalogo/469', 'https://sga.softexrecife.org.br/catalogo/468', 'https://sga.softexrecife.org.br/catalogo/471', 'https://sga.softexrecife.org.br/catalogo/467', 'https://sga.softexrecife.org.br/catalogo/464', 'https://sga.softexrecife.org.br/catalogo/463', 'https://sga.softexrecife.org.br/catalogo/462', 'https://sga.softexrecife.org.br/catalogo/461', 'https://sga.softexrecife.org.br/catalogo/460', 'https://sga.softexrecife.org.br/catalogo/459', 'https://sga.softexrecife.org.br/catalogo/457', 'https://sga.softexrecife.org.br/catalogo/455', 'https://sga.softexrecife.org.br/catalogo/453', 'https://sga.softexrecife.org.br/catalogo/452', 'https://sga.softexrecife.org.br/catalogo/450', 'https://sga.softexrecife.org.br/catalogo/449', 'https://sga.softexrecife.org.br/catalogo/446', 'https://sga.softexrecife.org.br/catalogo/444', 'https://sga.softexrecife.org.br/catalogo/448']


In [12]:
card_resposta = requests.get(cards_url[0])

In [13]:
card_html_text = card_resposta.text

cards_selector = parsel.Selector(card_html_text)

In [17]:
paginacao = html_selector.xpath("//ul[@class='pagination pagination']/li").getall()
len(paginacao)

14

In [103]:
img = cards_selector.xpath("//div[@class='col-contact']//div[@class='col-img']/img/@src").get()

In [104]:
img

'https://sga.softexrecife.org.br/rails/active_storage/blobs/redirect/eyJfcmFpbHMiOnsibWVzc2FnZSI6IkJBaHBBcTBLIiwiZXhwIjpudWxsLCJwdXIiOiJibG9iX2lkIn19--016e3f76c1882d03ad901548111ff1f213996737/1.png'

In [105]:
telefone = cards_selector.xpath("//div[@class='col-contact']//div[@class='col-content']/a[1]/text()").get()
email = cards_selector.xpath("//div[@class='col-contact']//div[@class='col-content']/a[2]/text()").get()
site = cards_selector.xpath("//div[@class='col-contact']//div[@class='col-content']/a[3]/text()").get()
endereco_card = cards_selector.xpath("//div[@class='col-contact']//div[@class='col-content']/a[4]/text()").get()


In [106]:
endereco_list = endereco_card.split()
endereco = ' '.join(endereco_list)
endereco

'Rua Dona Maria Cesar, 170 - sala 0203Recife/PE. CEP: 50030-140.'

In [107]:
padrao_cep = r'\b\d{5}-\d{3}\b'
cep_encontrado = re.findall(padrao_cep, endereco)[0]

In [108]:
print(telefone, email, site, cep_encontrado)

(81)99816-6331 liviac.atendeup@gmail.com https://atendeup.com/ 50030-140


In [109]:
nome_startup = cards_selector.xpath("//div[@class='col-details']/p/text()").get().strip()
descricao_startup = cards_selector.xpath("//div[@class='col-details']/textarea/text()").get().strip()
segmento_startup = cards_selector.xpath("//div[@class='col-details']/p[3]/text()").get()
modelo_startup = cards_selector.xpath("//div[@class='col-details']/p[5]/text()").get()

In [110]:
# Exemplo de dados para uma única startup
dados = {
    'Nome': [nome_startup],
    'Segmento': [segmento_startup],
    'Modelo': [modelo_startup],
    'Site': [site],
    'Email': [email],
    'Telefone': [telefone],
    'Endereço': [endereco],
    'Descrição': [descricao_startup]
}

df = pd.DataFrame(dados)

df.to_excel('teste.xlsx', index=False)

## Tentando automatizar

In [124]:
def trata_cpf(string):
    padrao_cep = r'\b\d{5}-\d{3}\b'
    cep_encontrado = re.findall(padrao_cep, string)[0]

    return cep_encontrado

def dados_startup(link):
    try:
        resposta = requests.get(link)
        cards_html = resposta.text
        cards_selector = parsel.Selector(cards_html)

        telefone = cards_selector.xpath("//div[@class='col-contact']//div[@class='col-content']/a[1]/text()").get()
        email = cards_selector.xpath("//div[@class='col-contact']//div[@class='col-content']/a[2]/text()").get()
        site = cards_selector.xpath("//div[@class='col-contact']//div[@class='col-content']/a[3]/text()").get()
        endereco = cards_selector.xpath("//div[@class='col-contact']//div[@class='col-content']/a[4]/text()").get().split()
        endereco = ' '.join(endereco)
        cep = trata_cpf(endereco)
        cidade = consultar_cep(cep).get('city')
        nome_startup = cards_selector.xpath("//div[@class='col-details']/p/text()").get().strip()
        descricao_startup = cards_selector.xpath("//div[@class='col-details']/textarea/text()").get().split()
        descricao_startup = ' '.join(descricao_startup)
        segmento_startup = cards_selector.xpath("//div[@class='col-details']/p[3]/text()").get()
        modelo_startup = cards_selector.xpath("//div[@class='col-details']/p[5]/text()").get()

        dados = {
            'Nome': nome_startup,
            'Segmento': segmento_startup,
            'Modelo': modelo_startup,
            'Site': site,
            'Email': email,
            'Telefone': telefone,
            'Endereço': endereco,
            'Cidade': cidade,
            'Descrição': descricao_startup,
        }

        return dados

                
    except Exception as E:
        print(f'Erro ao acessar o site: {E}, {link}')

def agrupa_links(links):
    url = 'https://sga.softexrecife.org.br/catalogo'
    cards_url = []
    for link in links:
        cards_url.append(url + '/' + os.path.basename(link))
    return  cards_url


# Pegando o ultimo id da página.

In [19]:
last_range = html_selector.xpath("//ul[@class='pagination pagination']/l3").get()
last_range



'18'

# Gerando o link para cada página e automatizando

In [113]:

def gera_links(last_range):

    links_paginas = []

    for pagina in range(1,int(last_range)+1):
        url = f'https://sga.softexrecife.org.br/catalogo?page={pagina}'
        links_paginas.append(url)
    
    return links_paginas


# Definindo função que vai percorrer todas as páginas e pegar os dados das startups

In [114]:
def percorre_startups(links_paginas):
    dados = []
    caminho_xpath = "//div[@class='catalog-content-main-company border cl-page-width rounded row d-flex align-items-center justify-content-center']/a/@href"
    for pagina in links_paginas:
        time.sleep(1)
        print(f'Pagina - {pagina}')
        resposta = requests.get(pagina)
        html_text = resposta.text
        html_selector = parsel.Selector(html_text)
        links = html_selector.xpath(caminho_xpath).getall()
        links = agrupa_links(links)
        for link in links:
            resposta = dados_startup(link)
            if resposta == None:
                continue
            else:
                dados.append(resposta)
        
    return dados


In [116]:
dados

for i in range(len(dados)):
    print(dados[i])

len(dados)

KeyError: 0

# Função para automatizar geração de planilha

In [89]:
def gera_planilha(dados, nome_planilha):
    df = pd.DataFrame(dados)
    df.to_excel(f'{nome_planilha}.xlsx', index=True)

In [128]:
links_paginas = gera_links(last_range)
dados = percorre_startups(links_paginas)
gera_planilha(dados,'Startups')

Pagina - https://sga.softexrecife.org.br/catalogo?page=1
Erro ao consultar o CEP 50030600: Status Code 404
Erro ao acessar o site: 'NoneType' object has no attribute 'get', https://sga.softexrecife.org.br/catalogo/449
Pagina - https://sga.softexrecife.org.br/catalogo?page=2
Pagina - https://sga.softexrecife.org.br/catalogo?page=3
Pagina - https://sga.softexrecife.org.br/catalogo?page=4
Pagina - https://sga.softexrecife.org.br/catalogo?page=5
Pagina - https://sga.softexrecife.org.br/catalogo?page=6
Pagina - https://sga.softexrecife.org.br/catalogo?page=7
Pagina - https://sga.softexrecife.org.br/catalogo?page=8
Pagina - https://sga.softexrecife.org.br/catalogo?page=9
Erro ao acessar o site: list index out of range, https://sga.softexrecife.org.br/catalogo/98
Erro ao acessar o site: list index out of range, https://sga.softexrecife.org.br/catalogo/117
Erro ao acessar o site: list index out of range, https://sga.softexrecife.org.br/catalogo/241
Erro ao acessar o site: list index out of ran

KeyboardInterrupt: 

In [129]:
dados

[{'Nome': 'ATENDE.UP',
  'Segmento': 'CommunityTech',
  'Modelo': 'SaaS ',
  'Site': 'https://atendeup.com/',
  'Email': 'liviac.atendeup@gmail.com',
  'Telefone': '(81)99816-6331',
  'Endereço': 'Rua Dona Maria Cesar, 170 - sala 0203Recife/PE. CEP: 50030-140.',
  'Descrição': 'A AtendeUp é uma plataforma inovadora que otimiza o atendimento empresarial, reunindo toda a equipe em um único número de WhatsApp. Com uma interface simples e fácil de usar, ela permite gerenciar os contatos de forma eficiente, garantindo respostas rápidas e personalizadas por meio de mensagens automáticas com textos, links, imagens, áudios, vídeos e documentos. Além disso, a AtendeUp possibilita organizar os contatos de maneira inteligente, categorizando-os por setores, etiquetas, canais e empresas, o que aumenta a produtividade e a eficiência no atendimento ao cliente.'},
 {'Nome': 'Septem Compliance Sistemas e Consultoria LTDA',
  'Segmento': 'GovTech',
  'Modelo': 'BPM',
  'Site': 'https://septemcompliance.